In [1]:
from pathlib import Path
import pandas as pd
import re

In [2]:
notebook_path = Path().resolve().parent.parent.parent
print("Current Notebook Path:", notebook_path)
data_path = notebook_path / "data" / "March_1"
print("Data Path:", data_path)

Current Notebook Path: C:\Users\Vignesh\Desktop\SDM
Data Path: C:\Users\Vignesh\Desktop\SDM\data\March_1


In [5]:
class_options = ["Class A", "Class B", "Class C", "Other"]
print("Select the class name:")
for i, option in enumerate(class_options, 1):
    print(f"{i}. {option}")
class_choice = int(input("Enter the number corresponding to your choice: ") or 1)
class_name = class_options[class_choice - 1] if 1 <= class_choice <= len(class_options) else "Other"

# Replace 'your_file.csv' with the actual path to your CSV file
file_path = f'{data_path}/raw_data.csv'

location = input("Enter the location of the header row (0-indexed): ")
ignore_rows = int(input("Enter the number of rows to ignore from the end: "))

location = int(location)
# Read the CSV file
df = pd.read_csv(file_path,header =location)
if ignore_rows > 0:
    df = df.iloc[:-ignore_rows]

# Display all columns in the dataframe
print("Columns in the CSV file:")
print(df.columns.tolist())

# Display the first few rows of the dataframe
df.head()


Select the class name:
1. Class A
2. Class B
3. Class C
4. Other
Columns in the CSV file:
['Student Name', 'Mobile', 'Staff', 'Networker', 'DOB', 'Age', 'Gender', 'Position', 'Aadhaar Name', 'Aadhaar DOB', 'Cummulative Value', '13 March 2025', '10 March 2025', '6 March 2025', '3 March 2025', '27 February 2025', '24 February 2025', '20 February 2025']


,Student Name,Mobile,Staff,Networker,DOB,Age,Gender,Position,Aadhaar Name,Aadhaar DOB,Cummulative Value,13 March 2025,10 March 2025,6 March 2025,3 March 2025,27 February 2025,24 February 2025,20 February 2025
0,Arun,44440570034,NaN,A.Nevis Jeyaseelan,NaN,NaN,NaN,NaN,NaN,NaN,7.0,O,O,O,O,O,O,O
1,Swapna Gopinath,8111024659,NaN,Aarti Vignesh,20-May-1998,26.0,Female,Believer,NaN,NaN,7.0,O,O,O,O,O,O,O
2,Gipsy,9952527607,NaN,Aarti Vignesh,27-Jan-2001,24.0,Female,Believer,NaN,NaN,4.0,X,O,O,X,O,O,X
3,Iniya .S.,6381641661,NaN,Abin C Paul,NaN,NaN,NaN,NaN,NaN,NaN,6.0,O,O,O,O,O,O,X
4,Mugesh,9025159504,NaN,Abin C Paul,15-Jun-2002,22.0,Male,Seminar Student,NaN,NaN,5.0,O,X,O,O,O,O,X


In [6]:
patterns = {
    "student_name": re.compile(r'name|student', re.IGNORECASE),
    "mobile_phone": re.compile(r'mobile|phone', re.IGNORECASE),
    "staff": re.compile(r'staff', re.IGNORECASE),
    "networker": re.compile(r'networker', re.IGNORECASE)
}

In [7]:
matched_columns = {}
for key, pattern in patterns.items():
    for column in df.columns:
        if pattern.search(column):
            matched_columns[key] = column
            break

In [8]:
print("Matched Columns:")
print(matched_columns)

Matched Columns:
{'student_name': 'Student Name', 'mobile_phone': 'Mobile', 'staff': 'Staff', 'networker': 'Networker'}


In [9]:
# Keep only matched columns
df = df[[col for col in matched_columns.values() if col in df.columns]]

# Normalize mobile numbers and determine country if a matching column exists
if "mobile_phone" in matched_columns:
    mobile_col = matched_columns["mobile_phone"]
    df[mobile_col] = df[mobile_col].astype(str).str.replace(r'\D', '', regex=True)  # Remove non-numeric characters
    
    # Determine country
    def get_country(phone):
        if len(phone) == 10:
            return "India"
        elif len(phone) == 9 and phone.startswith("7"):
            return "Sri Lanka"
        elif phone.startswith("94") and len(phone) == 11:
            return "Sri Lanka"
        else:
            return "Others"
    
    df["Country"] = df[mobile_col].apply(get_country)

In [10]:
df["Class"] = class_name
df.head(10)

,Student Name,Mobile,Staff,Networker,Country,Class
0,Arun,44440570034,NaN,A.Nevis Jeyaseelan,Others,Class A
1,Swapna Gopinath,8111024659,NaN,Aarti Vignesh,India,Class A
2,Gipsy,9952527607,NaN,Aarti Vignesh,India,Class A
3,Iniya .S.,6381641661,NaN,Abin C Paul,India,Class A
4,Mugesh,9025159504,NaN,Abin C Paul,India,Class A
5,Jenitha,8870943046,NaN,Abirami stephen,India,Class A
6,Darthy Freeda,8610569887,NaN,Agadeesh,India,Class A
7,Kowsalya,9361360680,NaN,Agadeesh,India,Class A
8,R Vembarasi,9585127433,NaN,Agadeesh,India,Class A
9,Praisey Beulah D,9843781388,NaN,Agadeesh,India,Class A


In [56]:
df.count()

Student Name    463
Mobile          463
Staff             0
Networker       177
Country         463
dtype: int64